# CS4042 Data Engineering Group Project

In [37]:
import pandas as pd
import numpy as np
import os

### Preprecessing

In [38]:
sources = ['Player-Advanced-Stats', 'Player-Per-Game-Stats', 'Player-Shooting-Stats', 'Player-Totals-Stats']

# Dictionary for each folder
advanced = {}
per_game = {}
shooting = {}
totals = {}

for src in sources:

    src_path = os.path.join('Datasets', src)

    for file in os.listdir(src_path):
        
        file_path = os.path.join(src_path, file)
        name = file[4:-4] # Name of file

        # Assign DataFrame to correct dict
        if name[-1] == 'd':
            advanced[name] = pd.read_csv(file_path)
        elif name[-1] == 'G':
            per_game[name] = pd.read_csv(file_path)
        elif name[-1] == 'g':
            shooting[name] = pd.read_csv(file_path)
        else:
            totals[name] = pd.read_csv(file_path)



In [39]:
a = advanced['2022-23-Player-Stats-Advanced'] # For Data Wrangler, DELET LATER
s = shooting['2022-23-Player-Stats-Shooting']
p = per_game['2022-23-Player-Stats-PG']
t = totals['2022-23-Player-Stats-Totals']

### Dirty Data - Ensure the quality of the pipeline

This section aims to create data quality issues within the tables to show that our pipeline can handle these concerns without problem. 

In [ ]:
# Adds 'junk' column and fills it with empty strings.
def junk_col(df, col_name='junk'):
    if df.columns[-1] != col_name:
        df[col_name] = ''

# Add a fake leauge average summary row
def avg_row(df):
    if df.empty:
        return
    
    new_row = {col: np.nan for col in df.columns}
    if len(df.columns) > 1:
        second_col = df.columns[1]
        new_row[second_col] = "League Average"
    
    new_index = df.index.max() + 1 if len(df.index) > 0 else 0
    df.loc[new_index] = new_row

# Randomly duplicate rows
def dup_rows(df, n_dups=10, random_state=42):
    if len(df) == 0:
        return
    
    rng = np.random.default_rng(random_state)
    idx_pool = df.index[:-1] if len(df) > 1 else df.index  
    
    if len(idx_pool) == 0:
        return
    
    n_dups = min(n_dups, len(idx_pool))
    dup_indices = rng.choice(idx_pool, size=n_dups, replace=False)

    for idx in dup_indices:
        new_index = df.index.max() + 1
        df.loc[new_index] = df.loc[idx]

# add extra spaces
def add_whitespace(df, columns=None):
    if columns is None:
        columns = df.select_dtypes(include=["object"]).columns

    for col in columns:
        df[col] = df[col].apply(lambda v: f"  {v}  " if isinstance(v, str) else v)

# Convert Numeric values to strings
def numeric_to_string(df, columns=None):
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns

    for col in columns:
        df[col] = df[col].apply(lambda v: f" {v} " if pd.notna(v) else v)


In [ ]:
def data_dirtier(advanced, per_game, shooting, totals, seed=42):
    for df in advanced.values():
        junk_col(df)
        dup_rows(df, random_state=seed)
        avg_row(df)
        add_whitespace(df)

    for df in per_game.values():
        junk_col(df)
        dup_rows(df, random_state=seed)
        avg_row(df)
        add_whitespace(df)

    for df in shooting.values():
        junk_col(df)
        dup_rows(df, random_state=seed)
        avg_row(df)
        add_whitespace(df)
        numeric_to_string(df)  

    for df in totals.values():
        junk_col(df)
        dup_rows(df, random_state=seed)
        avg_row(df)
        add_whitespace(df)


data_dirtier(advanced, per_game, shooting, totals, seed=42)



,Rk,Player,Age,Team,Pos,G,GS,MP,PER,TS%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards,Player-additional,junk
764,391.0,Kessler Edwards,22.0,BRK,SF,14.0,1.0,79.0,2.4,0.301,...,0.1,-0.2,-0.108,-6.7,-3.0,-9.7,-0.2,NaN,edwarke02,
765,60.0,Luguentz Dort,23.0,OKC,SF,74.0,73.0,2272.0,11.3,0.513,...,2.1,2.9,0.062,-2.0,-1.0,-3.0,-0.6,NaN,dortlu01,
766,121.0,Tyus Jones,26.0,MEM,PG,80.0,22.0,1940.0,16.0,0.546,...,2.3,5.7,0.141,1.2,0.3,1.6,1.7,6MOY-6,jonesty01,
767,484.0,Jamaree Bouyea,23.0,2TM,PG,5.0,0.0,71.0,6.5,0.504,...,0.1,0.0,0.013,-5.8,1.6,-4.2,0.0,NaN,bouyeja01,
768,NaN,League Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean Data - Fix Tables
This section aims to standardise the tables. Ensures there is only one header row, all datatypes are correct for columns, and removes redunant columns

In the case where there are two heading rows, merge them (only happens in shooting df)

In [42]:
def concatinate_headings(top_head, sub_head):
    heading = top_head + "_" + sub_head
    return heading

def clean_heading(heading, type):
    # Remove whitespace
    heading = heading.strip()

    # Replace spaces in text
    heading = heading.replace(" ", "_")

    # Remove trailing numbers in top_head
    if type == "top":
        if "." in heading:
            before, after = heading.rsplit(".", 1) # Splits at last "."
            if after.isdigit(): # if digits follow
                heading = before

    return heading

def logic(df):
    for column in df:
        # Break if there is only one heading
        if "Rk" in str(column):
            break
        # Skip if top heading doesn't have a value
        elif "Unnamed" in str(column):
            continue
        else:
            topheading = clean_heading(column, "top")
            subheading = clean_heading(df[column][0], "sub")
            output = concatinate_headings(topheading, subheading)
            df.loc[0, column] = output

# Apply changes
for df in shooting.values():
    logic(df)


Remove top headings and replace with concatinated headings done above (only happens in shooting)

In [43]:
def drop_irregular_headings(df):
    if df.columns[0] == "Unnamed: 0":
        df.columns = df.iloc[0]
        df.drop(index=0, inplace=True)
        df.reset_index(drop=True, inplace=True) # drop ensures previous index isn't repeated in the dataframe as a seperate column

# Apply changes
for df in shooting.values():
    drop_irregular_headings(df)

Ensure columns have correct data types (types only incorrect in shooting)
By default, that table's columns are all string objects

In [44]:
def check_type_is_digit(df):
    for column in df.columns:
        value = str(df[column][0]).strip()

        # Skip empty cells
        if value == None:
            continue

        if value == "":
            continue

        if value == "nan":
            continue

        # Convert strings to floats if they are a number or decimal
        try:
            float(value)
            df[column] = df[column].astype(float)
        except:
            continue

for df in shooting.values():
    check_type_is_digit(df)

Drop the final column as it doesn't contain relevant data

In [45]:
for df in advanced.values():
    if df.columns[-1] != "Awards":
        df.drop(df.columns[-1], axis=1, inplace=True)
for df in per_game.values():
    if df.columns[-1] != "Awards":
        df.drop(df.columns[-1], axis=1, inplace=True)
for df in shooting.values():
    if df.columns[-1] != "Awards":
        df.drop(df.columns[-1], axis=1, inplace=True)
for df in totals.values():
    if df.columns[-1] != "Awards":
        df.drop(df.columns[-1], axis=1, inplace=True)

Drop the final row as it doesn't contain relevant data

In [46]:
for df in advanced.values():
    if df.iloc[-1, 1] == "League Average":
        df.drop(df.index[-1], axis=0, inplace=True)
for df in per_game.values():
    if df.iloc[-1, 1] == "League Average":
        df.drop(df.index[-1], axis=0, inplace=True)
for df in shooting.values():
    if df.iloc[-1, 1] == "League Average":
        df.drop(df.index[-1], axis=0, inplace=True)
for df in totals.values():
    if df.iloc[-1, 1] == "League Average":
        df.drop(df.index[-1], axis=0, inplace=True)

### Clean Data - Making Data Analysis-Worthy

Check for duplicate rows and remove them

In [47]:
def remove_duplicates(df):
    df.drop_duplicates(inplace=True)

"""
Optionally, drop only by subset (eg. player, team).
This way if there is an error with the data, where the wrong team or player was placed in a cell, those will get dropped as well.
Even if other columns have different values.

def remove_duplicates(df):
    df.drop_duplicates(subset=["Player", "Team"], inplace=True)
"""

for df in advanced.values():
    remove_duplicates(df)
for df in per_game.values():
    remove_duplicates(df)
for df in shooting.values():
    remove_duplicates(df)
for df in totals.values():
    remove_duplicates(df)

Strip all data of whitespace

In [48]:
def strip_whitespace(df):
    df.iloc[0:] = df.iloc[0:].map(
        # Strips if value is a string object only
        lambda value: value.strip() if isinstance(value, str) else value
    )

for df in advanced.values():
    strip_whitespace(df)
for df in per_game.values():
    strip_whitespace(df)
for df in shooting.values():
    strip_whitespace(df)
for df in totals.values():
    strip_whitespace(df)

Convert common cell placeholders to NaN

In [49]:
def convert_to_nan(df):
    df.iloc[0:] = df.iloc[0:].map(
        # Convers to NaN if...
        lambda value: np.nan

        # Empty string
        if (isinstance(value, str) and value.strip() == "")

        # Other common placeholders
        or (isinstance(value, str) and "n/a" in str(value).lower())
        or (isinstance(value, str) and "null" in str(value).lower())
        or (isinstance(value, str) and str(value) == "?")
        or (isinstance(value, str) and "unknown" in str(value).lower())

        # Otherwise keep value
        else value
    )

for df in advanced.values():
    convert_to_nan(df)
for df in per_game.values():
    convert_to_nan(df)
for df in shooting.values():
    convert_to_nan(df)
for df in totals.values():
    convert_to_nan(df)

Remove any player whos played less than 5 games.

In [50]:
for df in advanced.values():
    df.drop(df[df['G'] < 5].index, inplace=True)
for df in per_game.values():
    df.drop(df[df['G'] < 5].index, inplace=True)
for df in shooting.values():
    df.drop(df[df['G'] < 5].index, inplace=True)
for df in totals.values():
    df.drop(df[df['G'] < 5].index, inplace=True)

KeyError: 'G'